In [1]:
import gym
import gym.spaces
import numpy as np

In [3]:
def check_policy_init(initial_policy):
    assert type(initial_policy) in (np.ndarray, np.matrix)
    assert np.allclose(initial_policy, 1. / n_actions)
    assert np.allclose(np.sum(initial_policy, axis=1), 1)
    print('Policy initialization: Ok!')

In [4]:
def check_generate_session_func(generation_func):
    s, a, r = generation_func(policy)
    assert type(s) == type(a) == list
    assert len(s) == len(a)
    assert type(r) in [float, np.float]
    print('Session generation function: Ok!')

In [5]:
def check_update_policy_func(update_func):
    elite_states, elite_actions = ([1, 2, 3, 4, 2, 0, 2, 3, 1], [0, 2, 4, 3, 2, 0, 1, 3, 3])
    new_policy = update_func(elite_states, elite_actions, 5, 6)

    assert np.isfinite(new_policy).all(), 'Your new policy contains NaNs or +-inf. Make sure you do not divide by zero.'
    assert np.all(new_policy >= 0), 'Your new policy should not have negative action probabilities'
    assert np.allclose(new_policy.sum(axis=-1), 1),  \
        'Your new policy should be a valid probability distribution over actions'

    reference_answer = np.array([
           [1.,  0.,  0.,  0.,  0.],
           [0.5,  0.,  0.,  0.5,  0.],
           [0.,  0.33333333,  0.66666667,  0.,  0.],
           [0.,  0.,  0.,  0.5,  0.5]])
    assert np.allclose(new_policy[:4, :5], reference_answer)
    print('Update policy function: Ok!')

In [6]:
def check_select_elites_func(select_elite_func):
    states_batch = [[1, 2, 3], [4, 2, 0, 2], [3, 1]]
    actions_batch = [[0, 2, 4], [3, 2, 0, 1], [3, 3]]
    rewards_batch = [3, 4, 5]

    test_result_0 = select_elite_func(states_batch, actions_batch, rewards_batch, percentile=0)
    test_result_40 = select_elite_func(states_batch, actions_batch, rewards_batch, percentile=30)
    test_result_90 = select_elite_func(states_batch, actions_batch, rewards_batch, percentile=90)
    test_result_100 = select_elite_func(states_batch, actions_batch, rewards_batch, percentile=100)

    assert np.all(test_result_0[0] == [1, 2, 3, 4, 2, 0, 2, 3, 1]) and \
        np.all(test_result_0[1] == [0, 2, 4, 3, 2, 0, 1, 3, 3]), \
        'For percentile 0 you should return all states and actions in chronological order'

    assert np.all(test_result_40[0] == [4, 2, 0, 2, 3, 1]) and \
        np.all(test_result_40[1] == [3, 2, 0, 1, 3, 3]), \
        'For percentile 30 you should only select states/actions from two first'

    assert np.all(test_result_90[0] == [3, 1]) and \
        np.all(test_result_90[1] == [3, 3]), \
        'For percentile 90 you should only select states/actions from one game'

    assert np.all(test_result_100[0] == [3, 1]) and \
        np.all(test_result_100[1] == [3, 3]), \
        'Please make sure you use >=, not >. Also double-check how you compute percentile.'
    print('Select elites function : Ok!')

In [7]:
def generate_session(policy, t_max=10**5):
    """
    Play game until end or for t_max ticks.
    :param policy: an array of shape [n_states,n_actions] with action probabilities
    :returns: list of states, list of actions and sum of rewards
    """
    states, actions = [], []
    total_reward = 0.

    s = env.reset()

    for t in range(t_max):
        # Choose action from policy
        # You can use np.random.choice() func
        # a = ?
        a = np.random.choice(n_actions, p=policy[s])

        # Do action `a` to obtain new_state, reward, is_done,
        new_s, r, is_done, _ = env.step(a)

        # Record state, action and add up reward to states, actions and total_reward accordingly.
        states.append(s)
        actions.append(a)
        total_reward += r

        # Update s for new cycle iteration
        s = new_s
        if is_done:
            break

    return states, actions, total_reward

In [8]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i][t]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    Please return elite states and actions in their original order
    [i.e. sorted by session number and timestep within session]

    If you're confused, see examples below. Please don't assume that states are integers (they'll get different later).
    """

    states_batch, actions_batch, rewards_batch = map(np.array, [states_batch, actions_batch, rewards_batch])

    # Compute reward threshold
    reward_threshold = np.percentile(rewards_batch, q=percentile)

    # Compute elite states using reward threshold
    elite_states = states_batch[rewards_batch >= reward_threshold]

    # Compute elite actions using reward threshold
    elite_actions = actions_batch[rewards_batch >= reward_threshold]

    elite_states, elite_actions = map(np.concatenate, [elite_states, elite_actions])

    return elite_states, elite_actions

In [9]:
def update_policy(elite_states, elite_actions, n_states, n_actions):
    """
    Given old policy and a list of elite states/actions from select_elites,
    return new updated policy where each action probability is proportional to

    policy[s_i,a_i] ~ #[occurences of si and ai in elite states/actions]

    Don't forget to normalize policy to get valid probabilities and handle 0/0 case.
    In case you never visited a state, set probabilities for all actions to 1./n_actions

    :param elite_states: 1D list of states from elite sessions
    :param elite_actions: 1D list of actions from elite sessions

    """
    new_policy = np.zeros([n_states, n_actions])

    for el_state, el_action in zip(elite_states, elite_actions):
        new_policy[el_state][el_action] += 1

    sum_array = new_policy.sum(axis=1)

    for s in range(0, n_states):
        if sum_array[s] == 0:
            new_policy[s] = np.full((1, n_actions), 1.0 / n_actions)
        else:
            new_policy[s] /= sum_array[s]

    return new_policy

In [24]:
def rl_cross_entropy():
    # Useful constants, all should be applied somewhere in your code
    n_sessions = 200  # generate n_sessions for analysis
    # чем больше n_sessions, тем намного медленнее спуск, но больше mean reward (+3 после 100 итераций и n_sessions = 500)
    percentile = 20  # take this percentage of 'elite' states/actions
    # чем меньше percentile, тем медленнее спуск и меньше mean reward (-161 после 100 итераций и percentile = 10)
    alpha = 0.3  # alpha-blending for policy updates
    # чем больше alpha, тем меньше mean reward (-4 после 100 итераций и alpha = 0.5)
    # сейчас mean reward примерно -0.5 после 100 итераций
    total_iterations = 100
    # чем больше total_iterations, тем чаще положительный mean reward, ниже theshold
    visualize = False
    log = []

    # Create random uniform policy
    policy = np.full([n_states, n_actions], 1.0 / n_actions)
    check_policy_init(policy)

    if visualize:
        import matplotlib.pyplot as plt
        plt.figure(figsize=[10, 4])

    for i in range(total_iterations):

        # Generate n_sessions for further analysis.
        sessions = [generate_session(policy) for _ in range(n_sessions)]

        states_batch, actions_batch, rewards_batch = zip(*sessions)

        # Select elite states & actions.
        elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

        # Update policy using elite_states, elite_actions.
        new_policy = update_policy(elite_states, elite_actions, n_states, n_actions)

        # Alpha blending of old & new policies for stability.
        policy = alpha * new_policy + (1 - alpha) * policy

        # Info for debugging
        mean_reward = np.mean(rewards_batch)
        threshold = np.percentile(rewards_batch, percentile)
        log.append([mean_reward, threshold])

        print('Iteration = %.0f, Mean Reward = %.3f, Threshold = %.3f' % (i, mean_reward, threshold))

        # Visualize training
        if visualize:

            plt.subplot(1, 2, 1)
            plt.plot(list(zip(*log))[0], label='Mean rewards', color='red')
            plt.plot(list(zip(*log))[1], label='Reward thresholds', color='green')

            if i == 0:
                plt.legend()
                plt.grid()

            plt.subplot(1, 2, 2)
            plt.hist(rewards_batch, range=[-990, +10], color='blue', label='Rewards distribution')
            plt.vlines([np.percentile(rewards_batch, percentile)], [0], [100], label='Percentile', color='red')

            plt.legend()
            plt.grid()

            plt.pause(0.1)
            plt.cla()

In [25]:
if __name__ == '__main__':
    # Create environment 'Taxi-v2'
    env = gym.make('Taxi-v2')
    env.reset()
    env.render()

    # Compute number of states for this environment
    n_states = env.observation_space.n
    # Compute number of actions for this environment
    n_actions = env.action_space.n

    print('States number = %i, Actions number = %i' % (n_states, n_actions))

    # Initialize policy - let's say random uniform
    policy = np.full((n_states, n_actions), 1.0 / n_actions)
    check_policy_init(policy)

    # Complete generate session function
    check_generate_session_func(generate_session)

    # Complete select elites function
    check_select_elites_func(select_elites)

    # Complete update policy function
    check_update_policy_func(update_policy)

    # Complete rl_cross_entropy()
    rl_cross_entropy()

    # Close environment when everything is done
    env.close()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

States number = 500, Actions number = 6
Policy initialization: Ok!
Session generation function: Ok!
Select elites function : Ok!
Update policy function: Ok!
Policy initialization: Ok!
Iteration = 0, Mean Reward = -780.755, Threshold = -839.000
Iteration = 1, Mean Reward = -773.370, Threshold = -830.000
Iteration = 2, Mean Reward = -742.790, Threshold = -830.000
Iteration = 3, Mean Reward = -743.320, Threshold = -804.800
Iteration = 4, Mean Reward = -719.885, Threshold = -803.000
Iteration = 5, Mean Reward = -704.940, Threshold = -794.000
Iteration = 6, Mean Reward = -708.430, Threshold = -785.000
Iteration = 7, Mean Reward = -703.175, Threshold = -785.000
Iteration = 8, Mean Reward = -690.130, Threshold = -785.000
Iteration = 9, Mean Reward = -680.200, Threshold = -767.000
Iteration = 10, Mean Reward = -677.305, Threshold = -767.000
Iteration = 11, Mean Reward = -659.430, Threshold = -749.400
Iteration

Iteration = 134, Mean Reward = -1.210, Threshold = -8.200
Iteration = 135, Mean Reward = -0.935, Threshold = -6.000
Iteration = 136, Mean Reward = -2.865, Threshold = -7.200
Iteration = 137, Mean Reward = -1.670, Threshold = -7.000
Iteration = 138, Mean Reward = 0.940, Threshold = -2.000
Iteration = 139, Mean Reward = -0.220, Threshold = -5.000
Iteration = 140, Mean Reward = 0.105, Threshold = -8.000
Iteration = 141, Mean Reward = 1.070, Threshold = -4.000
Iteration = 142, Mean Reward = 0.825, Threshold = -3.200
Iteration = 143, Mean Reward = -3.380, Threshold = -8.400
Iteration = 144, Mean Reward = -3.535, Threshold = -5.000
Iteration = 145, Mean Reward = -0.155, Threshold = -6.000
Iteration = 146, Mean Reward = -1.345, Threshold = -4.000
Iteration = 147, Mean Reward = -1.380, Threshold = -7.000
Iteration = 148, Mean Reward = -0.095, Threshold = -6.000
Iteration = 149, Mean Reward = -0.070, Threshold = -4.000
Iteration = 150, Mean Reward = -0.680, Threshold = -4.400
Iteration = 151, M